In [1]:
import snscrape.modules.twitter as snt
import pandas as pd
import re
import emoji
from datetime import date, datetime
from collections import Counter
import nltk

In [2]:
#Reading twitter dataset file for preprocessing

final=pd.read_csv("final.csv")
print(final.head(5))

   Unnamed: 0                                                url   
0           0  https://twitter.com/gand_oliver/status/1648233...  \
1           1  https://twitter.com/MirrorF1/status/1648233870...   
2           2  https://twitter.com/MotorsportWeek/status/1648...   
3           3  https://twitter.com/f1reader/status/1648233423...   
4           4  https://twitter.com/Racingnews365C/status/1648...   

                        date            id   
0  2023-04-18 07:56:10+00:00  1.650000e+18  \
1  2023-04-18 07:55:52+00:00  1.650000e+18   
2  2023-04-18 07:54:56+00:00  1.650000e+18   
3  2023-04-18 07:54:05+00:00  1.650000e+18   
4  2023-04-18 07:53:56+00:00  1.650000e+18   

                                                user  replyCount   
0  {'username': 'gand_oliver', 'id': 753267774017...           0  \
1  {'username': 'MirrorF1', 'id': 113622858, 'dis...           2   
2  {'username': 'MotorsportWeek', 'id': 27343080,...           0   
3  {'username': 'f1reader', 'id': 23044052

preprocessing starts here

In [3]:
#replacing "" with NA in retweetedTweets and quotedTweets columns
final.fillna({"retweetedTweet": "NA", "quotedTweet": "NA"}, inplace=True)

tokenization, stop words removal, stemming, lemmatization, calculation of tfidf starts here

In [4]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.sentiment.vader import SentimentIntensityAnalyzer

TwtwithoutAT=[]
sentimentScore=[]
lemm_tokens=[]
tfidf_matrices=[]
for tweets in final[['renderedContent'][0]]:
    #removing @
    tweets = re.sub("@+","",tweets) #@[A-Za-z0-9]+
    #removing hyperlinks
    tweets = re.sub("(?:\\@|http?\\://|https?\\://|www)\\S+","",tweets)
    #replacing # and _
    tweets = tweets.replace("#", "").replace("_", " ")
    #removing/replacing emojis
    tweets = emoji.replace_emoji(tweets,"")
    #removing spaces from start and end
    tweets = re.sub(r'\n','', tweets).strip()
    # Replace Unicode characters in the 'tweets' variable
    tweets = tweets.replace("\u2019", "'")
    tweets = tweets.replace("\u2018", "'")
    tweets = tweets.replace("\u2026", "...")
    tweets = tweets.replace("\u2013", "-")
    tweets = tweets.replace("\u201c", "\"")
    tweets = tweets.replace("\u201d", "\"")

    text = tweets

    # Tokenization
    tokens = word_tokenize(text)

    # Stop words removal
    stop_words = set(stopwords.words('english'))
    filtered_tokens = [token for token in tokens if token.lower() not in stop_words]

    # Stemming
    stemmer = PorterStemmer()
    stemmed_tokens = [stemmer.stem(token) for token in filtered_tokens]

    # Lemmatization
    lemmatizer = WordNetLemmatizer()
    lemmatized_tokens = [lemmatizer.lemmatize(token) for token in stemmed_tokens]

    # TF-IDF
    documents = [" ".join(lemmatized_tokens)]  # Creating a single document for simplicity
    tfidf_vectorizer = TfidfVectorizer()
    tfidf_matrix = tfidf_vectorizer.fit_transform(documents)
    
    #Appending the final output to the lists
    lemm_tokens.append(lemmatized_tokens)
    tfidf_matrices.append(tfidf_matrix)
    TwtwithoutAT.append(text)

    # Sentiment Analysis
    analyzer = SentimentIntensityAnalyzer()
    sentimentScore.append(analyzer.polarity_scores(text))

In [5]:
#determining Sentiment values in categorical form

SentimentVal=[]

for i in sentimentScore:
    if i["compound"]>0:
        SentimentVal.append("Positive")
    elif i["compound"]<0:
        SentimentVal.append("Negative")
    else:
        SentimentVal.append("Neutral")

In [6]:
final['RefinedContent']=TwtwithoutAT
final['lemmatized_tokens']=lemm_tokens
final['tfidf_matrix']=tfidf_matrices
final['SentimentValue']=SentimentVal

In [7]:
final.to_csv("Twt_Refined_Final.csv")